In [1]:
import numpy as np
import pandas as pd

import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")


In [2]:
# data directory

data_dir = '../input/optiver-realized-volatility-prediction/'

In [3]:
def wap1(df):
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1'])/(df['bid_size1'] + df['ask_size1'])
    return wap
def wap2(df):
    wap = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2'])/(df['bid_size2'] + df['ask_size2'])
    return wap

In [4]:
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff() 

In [5]:
def realized_volatility(series):
    return np.sqrt(np.sum(series**2))

In [6]:
def preprocessor_book(file_path):
    
    df = pd.read_parquet(file_path)
    
    #calculate return, and other book-related feautures
    df['wap1'] = wap1(df)
    df['log_return1'] = df.groupby('time_id')['wap1'].transform(log_return)
    
    df['wap2'] = wap2(df)
    df['log_return2'] = df.groupby('time_id')['wap2'].transform(log_return)
    
    df['wap_balance'] = abs(df['wap1'] - df['wap2'])/((df['wap1'] + df['wap2'])/2)
    
    df['price_spread1'] = (df['ask_price1'] - df['bid_price1']) / ((df['ask_price1'] + df['bid_price1'])/2)
    df['price_spread2'] = (df['ask_price2'] - df['bid_price2']) / ((df['ask_price2'] + df['bid_price2'])/2)
    
    df['bid_spread'] = (df['bid_price1'] - df['bid_price2'])/((df['bid_price1'] + df['bid_price2'])/2)
    
    
    df['ask_spread'] = (df['ask_price1'] - df['ask_price2'])/((df['ask_price1'] + df['ask_price2'])/2)
    
    df['total_volume1'] = df['ask_size1'] + df['bid_size1']
    df['total_volume2'] = df['ask_size2'] + df['bid_size2']
    
    df['volume_imbalance1'] = abs(df['ask_size1'] - df['bid_size1'])
    df['volume_imbalance2'] = abs(df['ask_size2'] - df['bid_size2'])

    
    #dict for aggregate
    create_feature_dict = {
        'log_return1':[realized_volatility],
        'log_return2':[realized_volatility],
        'wap_balance':[np.mean, np.std],
        'price_spread1':[np.mean, np.std],
        'price_spread2':[np.mean, np.std],
        'bid_spread':[np.mean, np.std],
        'ask_spread':[np.mean, np.std],
        'volume_imbalance1':[np.mean, np.std],
        'total_volume1':[np.mean, np.std],
        'volume_imbalance2':[np.mean, np.std],
        'total_volume2':[np.mean, np.std],  
        'wap1':[np.mean, np.std],
        'wap2':[np.mean, np.std],   
            }

    #####groupby / all seconds
    df_feature = pd.DataFrame(df.groupby(['time_id']).agg(create_feature_dict)).reset_index()
    
    df_feature.columns = ['_'.join(col) for col in df_feature.columns] #time_id is changed to time_id_
        
    
    last_seconds = [300, 200, 100]
    
    for second in last_seconds:
        second = 600 - second 
    
        df_feature_sec = pd.DataFrame(df.query(f'seconds_in_bucket >= {second}').groupby(['time_id']).agg(create_feature_dict)).reset_index()

        df_feature_sec.columns = ['_'.join(col) for col in df_feature_sec.columns] #time_id is changed to time_id_
     
        df_feature_sec = df_feature_sec.add_suffix('_' + str(second))

        df_feature = pd.merge(df_feature,df_feature_sec,how='left',left_on='time_id_',right_on=f'time_id__{second}')
        df_feature = df_feature.drop([f'time_id__{second}'],axis=1)
    
    
    
    
    #create row_id
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['time_id_'].apply(lambda x:f'{stock_id}-{x}')
    df_feature = df_feature.drop(columns = ['time_id_'])

    
    return df_feature

In [7]:
%%time
file_path = data_dir + "book_train.parquet/stock_id=100"
preprocessor_book(file_path)

CPU times: user 7.19 s, sys: 506 ms, total: 7.7 s
Wall time: 7.65 s


,log_return1_realized_volatility,log_return2_realized_volatility,wap_balance_mean,wap_balance_std,price_spread1_mean,price_spread1_std,price_spread2_mean,price_spread2_std,bid_spread_mean,bid_spread_std,...,total_volume1_std_500,volume_imbalance2_mean_500,volume_imbalance2_std_500,total_volume2_mean_500,total_volume2_std_500,wap1_mean_500,wap1_std_500,wap2_mean_500,wap2_std_500,row_id
0,0.004813,0.007202,0.000317,0.000253,0.000606,0.000175,0.000903,0.000229,0.000156,0.000116,...,194.700188,148.343284,125.638678,253.537313,144.529365,1.001899,0.000893,1.001888,0.000965,100-5
1,0.002436,0.002899,0.000204,0.000161,0.000451,0.000188,0.000704,0.000188,0.000126,0.000081,...,324.933991,517.846154,310.837667,789.846154,303.524720,1.001489,0.000157,1.001296,0.000144,100-11
2,0.004001,0.005286,0.000179,0.000124,0.000380,0.000142,0.000690,0.000189,0.000156,0.000086,...,201.118509,344.095890,473.804934,465.000000,559.327672,0.999856,0.000451,0.999735,0.000489,100-16
3,0.002743,0.003732,0.000227,0.000178,0.000464,0.000145,0.000736,0.000176,0.000143,0.000086,...,108.743729,99.636364,66.445010,207.236364,120.856965,1.000647,0.000186,1.000573,0.000465,100-31
4,0.002714,0.004039,0.000315,0.000231,0.000501,0.000197,0.000856,0.000219,0.000186,0.000129,...,110.241393,121.000000,114.442522,199.844444,142.696747,1.000259,0.000580,1.000134,0.000526,100-62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3824,0.003507,0.005707,0.000273,0.000173,0.000462,0.000134,0.000727,0.000160,0.000123,0.000069,...,95.452072,90.885057,93.237310,143.482759,123.303168,1.000369,0.000231,1.000405,0.000398,100-32751
3825,0.002800,0.003495,0.000222,0.000154,0.000385,0.000130,0.000644,0.000168,0.000108,0.000043,...,94.253826,77.289474,63.391920,210.131579,103.099577,0.999218,0.000106,0.999248,0.000165,100-32753
3826,0.003194,0.004489,0.000269,0.000205,0.000671,0.000263,0.001040,0.000282,0.000177,0.000101,...,175.965495,100.200000,47.830952,259.133333,122.245952,1.001306,0.000388,1.001342,0.000406,100-32758
3827,0.004297,0.005072,0.000193,0.000151,0.000553,0.000217,0.000999,0.000275,0.000200,0.000107,...,132.187524,70.835616,57.545637,257.136986,93.185048,1.006575,0.000382,1.006619,0.000362,100-32763


In [8]:
def preprocessor_trade(file_path):
    
    df = pd.read_parquet(file_path)

    df['total_size'] = df['size']*df['order_count']

    df['log_return'] = df.groupby('time_id')['price'].transform(log_return)


    agg_dictionary = {
        'log_return':[realized_volatility],
        'seconds_in_bucket':[pd.Series.nunique],
        'total_size':[np.sum, np.std]
    }

    df_feature = df.groupby('time_id').agg(agg_dictionary)
    
    df_feature = df_feature.reset_index()
    df_feature.columns = ['_'.join(col) for col in df_feature.columns]


    last_seconds = [300, 200, 100]
    
    for second in last_seconds:
        second = 600 - second 
    
        df_feature_sec = pd.DataFrame(df.query(f'seconds_in_bucket >= {second}').groupby(['time_id']).agg(agg_dictionary)).reset_index()

        df_feature_sec.columns = ['_'.join(col) for col in df_feature_sec.columns] #time_id is changed to time_id_
     
        df_feature_sec = df_feature_sec.add_suffix('_' + str(second))

        df_feature = pd.merge(df_feature,df_feature_sec,how='left',left_on='time_id_',right_on=f'time_id__{second}')
        df_feature = df_feature.drop([f'time_id__{second}'],axis=1)

    

    df_feature = df_feature.add_prefix('trade_')
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['trade_time_id_'].apply(lambda x:f'{stock_id}-{x}')
    df_feature = df_feature.drop(['trade_time_id_'],axis=1)
    
    return df_feature

    

    

    

    

In [9]:
%%time
file_path = data_dir + "trade_train.parquet/stock_id=0"
preprocessor_trade(file_path)

CPU times: user 3.93 s, sys: 70.2 ms, total: 4 s
Wall time: 3.97 s


,trade_log_return_realized_volatility,trade_seconds_in_bucket_nunique,trade_total_size_sum,trade_total_size_std,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_nunique_300,trade_total_size_sum_300,trade_total_size_std_300,trade_log_return_realized_volatility_400,trade_seconds_in_bucket_nunique_400,trade_total_size_sum_400,trade_total_size_std_400,trade_log_return_realized_volatility_500,trade_seconds_in_bucket_nunique_500,trade_total_size_sum_500,trade_total_size_std_500,row_id
0,0.002006,40,17696,967.263462,0.001308,21.0,9590.0,1056.348348,0.001121,16.0,5583.0,800.917263,0.000959,10.0,3974.0,942.223163,0-5
1,0.000901,30,4511,380.524999,0.000587,16.0,3530.0,492.972599,0.000510,11.0,3329.0,583.408823,0.000451,7.0,1646.0,447.527812,0-11
2,0.001961,25,10305,764.014561,0.001137,12.0,5953.0,911.436873,0.001048,10.0,5649.0,989.283854,0.000723,4.0,2395.0,506.590811,0-16
3,0.001561,15,14023,1501.255230,0.001089,9.0,12467.0,1798.239332,0.000802,3.0,3723.0,2043.735061,0.000327,2.0,3718.0,2462.145812,0-31
4,0.000871,22,13063,1212.715840,0.000453,11.0,10182.0,1629.244872,0.000395,6.0,1058.0,290.489701,0.000348,3.0,346.0,194.577320,0-62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3825,0.001519,52,23556,1042.824915,0.001162,35.0,16437.0,1099.569998,0.000911,28.0,10881.0,863.773948,0.000672,9.0,3440.0,570.545962,0-32751
3826,0.001411,28,40560,2991.794116,0.001066,12.0,19697.0,3096.337175,0.000765,6.0,12544.0,3584.805025,0.000268,2.0,601.0,423.556962,0-32753
3827,0.001521,36,28658,1650.383279,0.001242,22.0,17274.0,1829.128250,0.000875,13.0,6398.0,1294.267801,0.000349,2.0,39.0,23.334524,0-32758
3828,0.001794,53,13118,638.890523,0.001404,25.0,5674.0,606.151898,0.001070,16.0,5502.0,739.496123,0.000601,7.0,4188.0,1094.041089,0-32763


In [10]:
def feature_gen(list_stock_ids, is_train = True):

    def feature_gen_stock(stock_id):
        if is_train:
            file_path_book = data_dir + "book_train.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_train.parquet/stock_id=" + str(stock_id)
        else:
            file_path_book = data_dir + "book_test.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_test.parquet/stock_id=" + str(stock_id)
       
        df_stock = pd.merge(preprocessor_book(file_path_book),preprocessor_trade(file_path_trade),on='row_id',how='left')

        return df_stock

    df_list = []

    for stock_id in list_stock_ids:
        df_list.append(feature_gen_stock(stock_id))

    return pd.concat(df_list, ignore_index = True)

    
    
    

In [11]:
%%time
list_stock_ids = [0,1]
feature_gen(list_stock_ids, is_train = True)

CPU times: user 20.8 s, sys: 394 ms, total: 21.2 s
Wall time: 21.1 s


,log_return1_realized_volatility,log_return2_realized_volatility,wap_balance_mean,wap_balance_std,price_spread1_mean,price_spread1_std,price_spread2_mean,price_spread2_std,bid_spread_mean,bid_spread_std,...,trade_total_size_sum_300,trade_total_size_std_300,trade_log_return_realized_volatility_400,trade_seconds_in_bucket_nunique_400,trade_total_size_sum_400,trade_total_size_std_400,trade_log_return_realized_volatility_500,trade_seconds_in_bucket_nunique_500,trade_total_size_sum_500,trade_total_size_std_500
0,0.004499,0.006999,0.000386,0.000294,0.000852,0.000211,0.001177,0.000213,0.000175,0.000162,...,9590.0,1056.348348,0.001121,16.0,5583.0,800.917263,0.000959,10.0,3974.0,942.223163
1,0.001204,0.002476,0.000212,0.000155,0.000394,0.000157,0.000671,0.000200,0.000142,0.000148,...,3530.0,492.972599,0.000510,11.0,3329.0,583.408823,0.000451,7.0,1646.0,447.527812
2,0.002369,0.004801,0.000331,0.000246,0.000725,0.000164,0.001120,0.000295,0.000197,0.000170,...,5953.0,911.436873,0.001048,10.0,5649.0,989.283854,0.000723,4.0,2395.0,506.590811
3,0.002574,0.003637,0.000381,0.000248,0.000860,0.000280,0.001159,0.000366,0.000190,0.000200,...,12467.0,1798.239332,0.000802,3.0,3723.0,2043.735061,0.000327,2.0,3718.0,2462.145812
4,0.001894,0.003257,0.000255,0.000188,0.000397,0.000130,0.000697,0.000185,0.000191,0.000083,...,10182.0,1629.244872,0.000395,6.0,1058.0,290.489701,0.000348,3.0,346.0,194.577320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7655,0.003723,0.004996,0.000330,0.000262,0.000597,0.000154,0.000873,0.000179,0.000157,0.000118,...,9462.0,558.624344,0.001028,16.0,8103.0,611.268187,0.000753,9.0,1987.0,315.676566
7656,0.010829,0.012168,0.000399,0.000329,0.000922,0.000333,0.001205,0.000354,0.000158,0.000119,...,663483.0,25677.405812,0.003885,51.0,414009.0,29235.776399,0.002413,27.0,263596.0,38586.221361
7657,0.003135,0.004268,0.000243,0.000168,0.000648,0.000150,0.000920,0.000141,0.000141,0.000064,...,2791.0,224.497702,0.001355,7.0,1989.0,254.811321,0.000987,4.0,1357.0,333.999376
7658,0.003750,0.005773,0.000198,0.000154,0.000421,0.000201,0.000841,0.000254,0.000189,0.000100,...,37589.0,1189.994884,0.001684,37.0,24668.0,1249.704829,0.001448,23.0,14927.0,1466.782192


In [12]:
train = pd.read_csv(data_dir + 'train.csv')

train.head(n=10)

,stock_id,time_id,target
0,0,5,0.004136
1,0,11,0.001445
2,0,16,0.002168
3,0,31,0.002195
4,0,62,0.001747
5,0,72,0.004912
6,0,97,0.009388
7,0,103,0.004120
8,0,109,0.002182
9,0,123,0.002669


In [13]:
list_of_stocks = train.stock_id.unique()

In [14]:
%%time
df_train = feature_gen(list_stock_ids= list_of_stocks, is_train = True)

CPU times: user 20min 3s, sys: 35.8 s, total: 20min 38s
Wall time: 20min 27s


In [15]:
output_path = "df_train.csv"
df_train.to_csv(output_path, index=False)

In [16]:
df_train.head(n=10)

,log_return1_realized_volatility,log_return2_realized_volatility,wap_balance_mean,wap_balance_std,price_spread1_mean,price_spread1_std,price_spread2_mean,price_spread2_std,bid_spread_mean,bid_spread_std,...,trade_total_size_sum_300,trade_total_size_std_300,trade_log_return_realized_volatility_400,trade_seconds_in_bucket_nunique_400,trade_total_size_sum_400,trade_total_size_std_400,trade_log_return_realized_volatility_500,trade_seconds_in_bucket_nunique_500,trade_total_size_sum_500,trade_total_size_std_500
0,0.004499,0.006999,0.000386,0.000294,0.000852,0.000211,0.001177,0.000213,0.000175,0.000162,...,9590.0,1056.348348,0.001121,16.0,5583.0,800.917263,0.000959,10.0,3974.0,942.223163
1,0.001204,0.002476,0.000212,0.000155,0.000394,0.000157,0.000671,0.000200,0.000142,0.000148,...,3530.0,492.972599,0.000510,11.0,3329.0,583.408823,0.000451,7.0,1646.0,447.527812
2,0.002369,0.004801,0.000331,0.000246,0.000725,0.000164,0.001120,0.000295,0.000197,0.000170,...,5953.0,911.436873,0.001048,10.0,5649.0,989.283854,0.000723,4.0,2395.0,506.590811
3,0.002574,0.003637,0.000381,0.000248,0.000860,0.000280,0.001159,0.000366,0.000190,0.000200,...,12467.0,1798.239332,0.000802,3.0,3723.0,2043.735061,0.000327,2.0,3718.0,2462.145812
4,0.001894,0.003257,0.000255,0.000188,0.000397,0.000130,0.000697,0.000185,0.000191,0.000083,...,10182.0,1629.244872,0.000395,6.0,1058.0,290.489701,0.000348,3.0,346.0,194.577320
5,0.007902,0.010336,0.000650,0.000461,0.001637,0.000403,0.001993,0.000506,0.000115,0.000140,...,6765.0,716.695642,0.002257,10.0,3428.0,476.698554,0.001159,2.0,1181.0,833.678895
6,0.010034,0.014493,0.000668,0.000481,0.001665,0.000445,0.002155,0.000425,0.000267,0.000278,...,4363.0,613.602795,0.002205,15.0,4175.0,720.175739,0.001566,7.0,3164.0,1025.689362
7,0.005331,0.006557,0.000493,0.000346,0.001039,0.000367,0.001419,0.000336,0.000133,0.000162,...,5391.0,1688.280944,0.000888,7.0,5372.0,2018.090257,0.000389,2.0,5.0,2.121320
8,0.001797,0.003536,0.000202,0.000163,0.000445,0.000146,0.000722,0.000214,0.000123,0.000098,...,2336.0,287.976169,0.000482,13.0,2011.0,346.635828,0.000357,7.0,437.0,110.217750
9,0.003273,0.005989,0.000243,0.000154,0.000468,0.000133,0.000749,0.000183,0.000121,0.000099,...,9092.0,629.201516,0.000654,8.0,3601.0,585.154299,0.000426,5.0,1489.0,490.057344


In [17]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 428932 entries, 0 to 428931
Columns: 113 entries, log_return1_realized_volatility to trade_total_size_std_500
dtypes: float32(36), float64(76), object(1)
memory usage: 310.9+ MB


In [18]:
df_train.tail()

,log_return1_realized_volatility,log_return2_realized_volatility,wap_balance_mean,wap_balance_std,price_spread1_mean,price_spread1_std,price_spread2_mean,price_spread2_std,bid_spread_mean,bid_spread_std,...,trade_total_size_sum_300,trade_total_size_std_300,trade_log_return_realized_volatility_400,trade_seconds_in_bucket_nunique_400,trade_total_size_sum_400,trade_total_size_std_400,trade_log_return_realized_volatility_500,trade_seconds_in_bucket_nunique_500,trade_total_size_sum_500,trade_total_size_std_500
428927,0.003691,0.005876,0.000361,0.000273,0.000878,0.000235,0.001171,0.000227,0.000091,0.000115,...,2658.0,263.575639,0.001067,13.0,1665.0,283.245024,0.000996,7.0,440.0,148.810202
428928,0.004104,0.004991,0.000295,0.000228,0.000706,0.000228,0.000974,0.000233,0.000126,0.000132,...,8882.0,856.141091,0.001388,13.0,1054.0,220.307536,0.001239,10.0,1047.0,249.051110
428929,0.003117,0.006020,0.000393,0.000306,0.000739,0.000240,0.001119,0.000258,0.000189,0.000198,...,10978.0,790.588219,0.001413,14.0,3193.0,277.635473,0.000742,2.0,724.0,497.803174
428930,0.003661,0.005362,0.000230,0.000176,0.000530,0.000172,0.000806,0.000204,0.000142,0.000115,...,25683.0,1146.579745,0.001117,27.0,15138.0,933.604759,0.000759,14.0,6187.0,889.408919
428931,0.002092,0.003037,0.000216,0.000154,0.000432,0.000125,0.000699,0.000171,0.000109,0.000066,...,15056.0,1058.735445,0.000478,10.0,10693.0,1094.067340,0.000364,5.0,3368.0,943.234488
